<a href="https://colab.research.google.com/github/Srish0218/SarcasmRudeModel/blob/main/SarcasmRudeModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas tqdm regex scipy tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 5.8 MB/s eta 0:00:00


In [ ]:
! pip install openai==0.28

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 1.8 MB/s eta 0:00:00


In [ ]:
import openai
import pandas as pd
import ast
import tiktoken
from scipy import spatial
import time
import regex as re
from tqdm import tqdm
import json

In [ ]:
tqdm.pandas()


In [ ]:
tick = time.time()


In [ ]:
df = pd.read_excel('OutPut Details - Cred- Parameter.xlsx', sheet_name='Transcript Chat')


In [ ]:
df.columns

Index(['id', 'request_id', 'Conversation ID', 'transcript', 'speaker',
       'sentiment', 'sentiment_score', 'starttime', 'Endtime', 'Holddiff',
       'Dear_Air_short', 'Dear_Air_long'],
      dtype='object')

In [ ]:
openai.api_key =


In [ ]:
if not all(col in df.columns for col in ['Conversation ID', 'speaker', 'transcript']):
    raise ValueError("The dataset must include ['Conversation ID', 'speaker', 'transcript']")


In [ ]:
rules = """
Follow below instructions strictly to check if the agent is being rude with the customer in the chat conversation.

You need to figure out if the agent was rude at any point in time during the conversation and give deterministic
and standard output every time for the same prompt.
Please mark the output as "Yes" if you find the conversation rude else "No".
Also give a reason why you have marked it as Yes or No.
For example, if you are marking it as Yes, then give the exact rude statement used by the agent in the chat conversation.
"""

example = """
Give Output strictly in the following JSON format after the explanation:

{
    "audit_outcome": (Yes or No),
    "reasons": [reason]
}
"""

In [ ]:
def get_prompt(TICKET_ID, audit_rules, example_out, df):
    single_chat = df[df['request_id'] == TICKET_ID]
    single_chat.reset_index(inplace=True)
    single_chat = single_chat[['speaker', 'transcript']]

    first_occurance = -1
    second_occurance = -1

    for index, row in single_chat.iterrows():
        if 'has joined the conversation' in row['transcript']:
            if first_occurance == -1:
                first_occurance = index
            else:
                second_occurance = index
                break

    if second_occurance != -1:
        only_first_agent_single_chat = single_chat.iloc[0:second_occurance]
    else:
        only_first_agent_single_chat = single_chat

    convo = ''
    for i in range(len(only_first_agent_single_chat)):
        if only_first_agent_single_chat.iloc[i]['speaker'] == 'bot':
            continue
        if 'assisting you today' in only_first_agent_single_chat.iloc[i]['transcript']:
            continue
        if 'Welcome to' in only_first_agent_single_chat.iloc[i]['transcript']:
            continue
        speaker = 'Agent' if only_first_agent_single_chat.iloc[i]['speaker'] == 0 else 'Customer'
        convo += f"{speaker}: {only_first_agent_single_chat.iloc[i]['transcript']}\n"

    prompt = f"Audit the given chat based on the audit rules and chat conversation given below\n\n\nBelow is a chat conversation between customer care agent and a customer\n\n```\n{convo}\n\n```\n{audit_rules}{example_out}"
    return prompt

In [ ]:
def get_response(prompt):
    messages = [
        {"role": "system", "content": "Imagine you are the best customer experience quality auditor in a company. Your primary responsibility is to comprehensively and contextually evaluate the chat transcript between the agent and the customer."},
        {"role": "user", "content": prompt},
    ]

    try:
        response = openai.ChatCompletion.create(
            model='gpt-3.5-turbo',
            messages=messages,
            temperature=0
        )
        response_message = response["choices"][0]["message"]["content"]
    except Exception as e:
        print(f"Error: {e}")
        response_message = '''
        {
            "audit_outcome": "Error",
            "reasons": []
        }
        '''

    return response_message

In [ ]:
ticket_list = list(df['request_id'].unique())


In [ ]:
res = pd.DataFrame(ticket_list, columns=['request_id'])


In [ ]:
res['JSON_OUT'] = res['request_id'].progress_apply(lambda x: get_response(get_prompt(x, rules, example, df)))


100%|██████████| 218/218 [04:03<00:00,  1.12s/it]


In [ ]:
def ea_out(x):
    try:
        return json.loads(x)['audit_outcome']
    except Exception as e:
        print(f"Error in ea_out: {e}")
        return 'Error'

def ea_reasons(x):
    try:
        return json.loads(x)['reasons']
    except Exception as e:
        print(f"Error in ea_reasons: {e}")
        return ['Error']

def remove_extra_text(x):
    only_json = ''
    read = False
    for c in x:
        if c == '{':
            read = True
        if read:
            only_json += c
        if c == '}':
            break
    return only_json

In [ ]:
res['Output'] = res['JSON_OUT'].apply(lambda x: ea_out(remove_extra_text(x)))
res['Evidence'] = res['JSON_OUT'].apply(lambda x: ea_reasons(remove_extra_text(x)))

Error in ea_out: Invalid control character at: line 3 column 605 (char 634)
Error in ea_reasons: Invalid control character at: line 3 column 605 (char 634)


In [ ]:
res.to_csv('Rude_Out.csv', index=False)


In [ ]:
SarcasmRude = res
Main = pd.read_excel('CRED_organised_result_12June10am.xlsx')

merged_df = pd.merge(Main,SarcasmRude,  on='request_id')


In [ ]:
merged_df.columns

Index(['request_id', 'apology_found', 'total_apology_count',
       'apology_evidence', 'empathy_found', 'total_empathy_count',
       'empathy_evidence', 'hold_request_found', 'total_hold_count',
       'hold_evidence', 'failure_to_assure', 'failure_to_assure_count',
       'failure_to_assure_evidence', 'rude_found', 'rude_evidence',
       'total_rude_behavior_count', 'sarcasm_prediction', 'sarcasm_evidence',
       'sarcasm_count', 'english_customer_count', 'hindi_customer_count',
       'english_agent_count', 'hindi_agent_count', 'language_switch',
       'JSON_OUT', 'Output', 'Evidence'],
      dtype='object')

In [ ]:
merged_df = merged_df.drop(columns=['JSON_OUT', 'rude_found', 'rude_evidence',
       'total_rude_behavior_count', 'sarcasm_prediction', 'sarcasm_evidence',
       'sarcasm_count'])
merged_df = merged_df.rename(columns={'Output': 'Sarcasm_rude_behaviour', 'Evidence': 'Sarcasm_rude_behaviour_evidence'})


In [ ]:
def update_evidence(row):
    if row['Sarcasm_rude_behaviour'] == 'Yes':
        return row['Sarcasm_rude_behaviour_evidence']
    else:
        return 'No evidence for sarcasm and rude behavior found'

merged_df['Sarcasm_rude_behaviour_evidence'] = merged_df.apply(update_evidence, axis=1)